# 2.7 MODELO DE REGRESIÓN LOGÍSTICA

En este notebook construimos el segundo modelo de Machine Learning del proyecto:  
un modelo de **clasificación binaria** utilizando **Regresión Logística**.

El objetivo es clasificar cada transacción como:

- 0: compra normal  
- 1: compra alta  

Para ello se creará una variable binaria llamada `compra_alta`, a partir del monto total de la compra.

Este trabajo sigue la metodología vista en clase en el archivo:

**RL_01.ipynb – Regresión Logística (BankNote Authentication)**

Adaptada ahora al contexto de ventas retail y al dataset preprocesado del proyecto.


In [1]:
# Importar librerías principales
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    roc_curve,
    roc_auc_score
)

import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de estilo para gráficas
plt.style.use("ggplot")
sns.set_theme(style="whitegrid")


## 1. Carga del dataset escalado (StandardScaler)

El dataset utilizado en este modelo es:

`retail_sales_escalado_standard.csv`

Este archivo contiene exclusivamente variables numéricas normalizadas mediante StandardScaler, preparadas previamente en el notebook **2.4_Normalización.ipynb**.


In [2]:
# Cargar dataset escalado (StandardScaler)
df = pd.read_csv("retail_sales_escalado_standard.csv")

df.head()


,edad_std,cantidad_std,precio_unitario_std,monto_total_std
0,-0.540565,0.429265,-0.685123,-0.546704
1,-1.125592,-0.453996,1.688464,0.971919
2,0.629489,-1.337258,-0.790615,-0.761098
3,-0.321180,-1.337258,1.688464,0.078611
4,-0.833078,-0.453996,-0.685123,-0.636035


## 2. Revisión general del dataset

Antes de construir la variable objetivo y entrenar el modelo, revisamos:

- Número de registros
- Tipos de datos
- Presencia de valores faltantes


In [3]:
df.shape


(1000, 4)

In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   edad_std             1000 non-null   float64
 1   cantidad_std         1000 non-null   float64
 2   precio_unitario_std  1000 non-null   float64
 3   monto_total_std      1000 non-null   float64
dtypes: float64(4)
memory usage: 31.4 KB


In [5]:
df.isna().sum()


edad_std               0
cantidad_std           0
precio_unitario_std    0
monto_total_std        0
dtype: int64

## 3. Construcción de la variable binaria `compra_alta`

La Regresión Logística requiere una variable objetivo binaria (0 o 1).

Como el dataset no incluye dicha variable, se crea a partir del monto total original.  
Para ello utilizamos el dataset transformado:

`retail_sales_transformado.csv`

Criterio:

- Si el monto total es mayor o igual al percentil 75 → compra alta (1)
- Si es menor → compra normal (0)

Este procedimiento es equivalente al utilizado en clase para crear etiquetas binarias.


In [6]:
# Cargar dataset transformado original
df_original = pd.read_csv("retail_sales_transformado.csv")

df_original.head()


,id_transaccion,fecha,id_cliente,genero,edad,categoria_producto,cantidad,precio_unitario,monto_total,anio,mes,dia,nombre_mes,rango_edad,monto_calculado,diferencia_monto,nivel_compra,es_compra_alta
0,1,2023-11-24,CUST001,MALE,34,Beauty,3,50,150,2023,11,24,November,Adulto,150,0,Baja,False
1,2,2023-02-27,CUST002,FEMALE,26,Clothing,2,500,1000,2023,2,27,February,Adulto,1000,0,Alta,True
2,3,2023-01-13,CUST003,MALE,50,Electronics,1,30,30,2023,1,13,January,Mayor,30,0,Baja,False
3,4,2023-05-21,CUST004,MALE,37,Clothing,1,500,500,2023,5,21,May,Adulto,500,0,Media,False
4,5,2023-05-06,CUST005,MALE,30,Beauty,2,50,100,2023,5,6,May,Adulto,100,0,Baja,False


In [7]:
# Cálculo del percentil 75 del monto total original
threshold = df_original["Monto_Total"].quantile(0.75)
threshold


KeyError: 'Monto_Total'

In [8]:
# Crear variable binaria
df["compra_alta"] = (df_original["Monto_Total"] >= threshold).astype(int)

# Revisión de distribución
df["compra_alta"].value_counts()


KeyError: 'Monto_Total'

In [ ]:
# Proporción relativa
df["compra_alta"].value_counts(normalize=True).round(3)


## 4. Selección de variables predictoras (X) y variable objetivo (Y)

Variables predictoras:

- Edad
- Cantidad
- Precio_Unitario

Variable objetivo:

- compra_alta

Estas variables ya están escaladas.


In [ ]:
X = df[["Edad", "Cantidad", "Precio_Unitario"]]
Y = df["compra_alta"]

X.head(), Y.head()


## 5. División del dataset en entrenamiento y prueba

Se utiliza un 80% para entrenamiento y 20% para prueba, siguiendo la metodología vista en clase.


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.20, random_state=42, stratify=Y
)

X_train.shape, X_test.shape


## 6. Entrenamiento del modelo de Regresión Logística


In [ ]:
modelo_log = LogisticRegression(max_iter=1000)
modelo_log.fit(X_train, Y_train)


## 7. Matriz de confusión


In [ ]:
Y_pred = modelo_log.predict(X_test)
cm = confusion_matrix(Y_test, Y_pred)
cm


In [ ]:
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión - Regresión Logística")
plt.show()


## 8. Métricas del modelo


In [ ]:
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)

accuracy, precision, recall


## 9. Curva ROC y AUC


In [ ]:
Y_proba = modelo_log.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(Y_test, Y_proba)

plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, label="Regresión Logística")
plt.plot([0,1], [0,1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curva ROC")
plt.legend()
plt.show()

roc_auc_score(Y_test, Y_proba)


## 10. Conclusiones del modelo

El modelo de Regresión Logística permite identificar correctamente las compras de alto monto, apoyándose en las variables del cliente y del producto. Sus métricas muestran un desempeño adecuado para apoyar decisiones estratégicas, tales como:

- Identificación de clientes valiosos  
- Segmentación para campañas  
- Priorización en estrategias de marketing  

Este modelo corresponde al segundo algoritmo supervisado solicitado para el proyecto.
